## New Schematics

In this notebook, we will be experimenting reading from schematics from 1.13+. This is when Minecraft changed its ID format, I want to see how we can handle this to be able to use new blocks in our models.

### Reading the Schematics

I will be using the `nbtschematic` library to read the schematics. This library is able to read the new format, but it does not have a way to write the new format. 

In [2]:
import nbtlib
import json

# load the schematic
sf = nbtlib.load('../../schematics/random/newblocks.schem')
# sf['Schematic'].keys()
# sf['Schematic']['Palette']
def nbt_to_dict(nbt):
    if isinstance(nbt, nbtlib.Compound):
        return {k: nbt_to_dict(v) for k, v in nbt.items()}
    elif isinstance(nbt, nbtlib.List):
        return [nbt_to_dict(v) for v in nbt]
    elif isinstance(nbt, nbtlib.ByteArray):
        return [nbt_to_dict(v) for v in nbt]
    elif isinstance(nbt, nbtlib.IntArray):
        return [nbt_to_dict(v) for v in nbt]
    else:
        return nbt

        

schematic_dict = nbt_to_dict(sf)
schematic_json = json.dumps(schematic_dict)
with open('newblocks.json', 'w') as f:
    f.write(schematic_json)

### Comparing my Schematics

I created a Java class to save schematics in the v2 format. I will be comparing the schematics I create `WorldEdit` to see if there are any differences.

In [3]:
with open('../../schematics/random/my_schematic.json') as new_schem:
    new_schem = json.load(new_schem)

for key in schematic_dict['Schematic'].keys():
    if key in new_schem['Schematic']:
        print(key, 'in my schematic')
    else:
        print(key, 'not in new_schem')

PaletteMax in my schematic
Palette in my schematic
Version in my schematic
Length in my schematic
Metadata in my schematic
Height in my schematic
DataVersion in my schematic
BlockData in my schematic
BlockEntities in my schematic
Width in my schematic
Offset in my schematic


### Palette and BlockData

The schematic format uses a palette to store the blocks. This is a list of all the blocks in the schematic. The actual position of the blocks is stored in BlockData. A list of indexes. This is useful as it allows us to store repeating blocks in a more efficient way.

In [36]:
def compare_sf_dicts(sf1, sf2):
    equal = True
    
    for key in sf1['Schematic'].keys():
        if key in sf2['Schematic']:
            print(key, 'in my schematic')
        else:
            print(key, 'not in new_schem')

    for block in sf1['Schematic']['Palette']:
        if block not in sf2['Schematic']['Palette']:
            print(block, 'not in sf1')
            equal = False
            continue

        position = int(sf1['Schematic']['Palette'][block])
        indexes_in_block = [i for i, pos in enumerate(sf1['Schematic']['BlockData']) if pos == position]

        indexes_in_block_new = [i for i, pos in enumerate(sf1['Schematic']['BlockData']) if pos == position]

        if len(indexes_in_block) != len(indexes_in_block_new):
            print('not the same amount of blocks', block, 'old:', len(indexes_in_block), 'new:', len(indexes_in_block_new))
            equal = False
            continue

        for a, b in zip(indexes_in_block, indexes_in_block_new):
            if a != b:
                print('not in same position', block, indexes_in_block, indexes_in_block_new)
                equal = False
                break

    return equal

if compare_sf_dicts(schematic_dict, new_schem):
    print('equal')
        

PaletteMax in my schematic
Palette in my schematic
Version in my schematic
Length in my schematic
Metadata in my schematic
Height in my schematic
DataVersion in my schematic
BlockData in my schematic
BlockEntities in my schematic
Width in my schematic
Offset in my schematic
equal


### NBT

I made some changes to my Schematic so it serializes to NBT (not JSON). This is because the `.schematic` standard uses NBT. 

In [66]:
test_1 = nbtlib.load('../../../MClassifierServer/server/plugins/WorldEdit/schematics/new_try.schematic')
test_1

<File '': Compound({'DataVersion': Int(3465), 'PaletteMax': Int(17), 'Width': Short(2), 'BlockData': ByteArray([Byte(0), Byte(1), Byte(2), Byte(3), Byte(4), Byte(5), Byte(6), Byte(7), Byte(8), Byte(9), Byte(10), Byte(11), Byte(12), Byte(13), Byte(12), Byte(12), Byte(12), Byte(13), Byte(12), Byte(12), Byte(12), Byte(14), Byte(15), Byte(16)]), 'Height': Short(2), 'Length': Short(6), 'Palette': Compound({'minecraft:purpur_block': Int(4), 'minecraft:chiseled_polished_blackstone': Int(2), 'minecraft:red_wool': Int(7), 'minecraft:clay': Int(1), 'minecraft:brown_wool': Int(9), 'minecraft:oak_leaves[distance=2,persistent=true,waterlogged=false]': Int(6), 'minecraft:cherry_leaves[distance=1,persistent=true,waterlogged=false]': Int(8), 'minecraft:dirt': Int(11), 'minecraft:oak_log[axis=y]': Int(10), 'minecraft:redstone_wire[east=side,north=none,power=0,south=none,west=side]': Int(15), 'minecraft:sea_lantern': Int(5), 'minecraft:air': Int(12), 'minecraft:packed_ice': Int(0), 'minecraft:end_stone'

### Yay it works!

I was able to get my schematics file to work.

In [2]:
import json

with open('newblocks.json', 'r') as f:
    json_data = json.load(f)
    json_string = json.dumps(json_data)

 

'{"Schematic": {"PaletteMax": 17, "Palette": {"minecraft:redstone_wire[east=side,north=none,power=0,south=none,west=side]": 15, "minecraft:sea_lantern": 5, "minecraft:oak_log[axis=y]": 10, "minecraft:brown_wool": 9, "minecraft:end_stone": 3, "minecraft:redstone_wire[east=none,north=side,power=0,south=side,west=none]": 14, "minecraft:dirt": 11, "minecraft:redstone_wire[east=none,north=side,power=0,south=none,west=side]": 16, "minecraft:clay": 1, "minecraft:air": 12, "minecraft:chiseled_polished_blackstone": 2, "minecraft:oak_leaves[distance=2,persistent=true,waterlogged=false]": 6, "minecraft:redstone_wire[east=none,north=none,power=0,south=none,west=none]": 13, "minecraft:purpur_block": 4, "minecraft:packed_ice": 0, "minecraft:red_wool": 7, "minecraft:cherry_leaves[distance=1,persistent=true,waterlogged=false]": 8}, "Version": 2, "Length": 6, "Metadata": {"WEOffsetX": 9, "WEOffsetY": -2, "WEOffsetZ": -1}, "Height": 2, "DataVersion": 3465, "BlockData": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,